In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 26 16:06:53 2020

@author: Fernando Montenegro
"""
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as pl
import random

from gurobipy import *

m=Model("vacunas")
#Sets
K=4 #macro-divisions of the region under study
L=[[1,2,3,4,5],[6,7,8,9,10],[11,12,13,14,15],[16,17,18,19,20]] #Set of zones of the macro-division
T=20 #Set of maximum periods to be planned 
P=3 #Set of population groups
I=4 #Set of fixed installations for vaccination 
J=5 #mobile vaccination center set up
delta=0.1#Minimum percentage of coverage with mobile centers


f1max=7670
f1min=7012

f2max=35000
f2min=4900

H=[[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
   [0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0],
   [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1]] #Set of areas that can be served in the same hospital

MM=1000000# 
amb=[350*random.uniform(0.99, 1.11) for i in range(T)]#Cost of temporary center
print(amb)

N={1:[1,2,3],2:[1,2,4],3:[1,3,4,5],4:[2,3,4,5],5:[3,4,5],
   6:[6,7,8],7:[6,7,9],8:[6,9,8,10],9:[7,8,9,10],10:[8,9,10],
   11:[11,12,13],12:[11,12,13,14],13:[11,12,13,15],14:[12,14,15],15:[13,14,15],
   16:[16,17,19],17:[16,17,18,19],18:[17,18,19,20],19:[17,18,19],20:[18,19,20]}

CK=[x for x in range(1,K+1)]
CT=[x for x in range(1,T+1)]
CP=[x for x in range(1,P+1)]
CI=[x for x in range(1,I+1)]
CJ=[x for x in range(1,J+1)]


#parameters
Po= [[10,220,120],[10,120,153],[20,190,210],[250,239,220],[250,239,220],
    [160,230,144],[164,121,167],[234,218,300],[276,239,212],[301,239,240],
      [182,100,146],[212,200,253],[198,310,215],[145,212,321],[255,249,250],
    [234,321,214],[241,208,145],[122,145,209],[234,145,271],[169,181,161]] #demand for vaccines in macrodivision by population groups


C=[150,150,150,150] #vaccination capacity of fixed centerss
D=[37 for i in range(J)] #vaccination capacity of mobile centers

A=[1000,1000,1000,1000,1000,
   1000,1000,1000,1000,1000,
   1000,1000,1000,1000,1000,
   1000,1000,1000,1000,1000] #9436 
    #supply capacity



rg=[[0.21,0.51,0.73], [0.22,0.53,0.74],[0.24,0.54,0.75], [0.25,0.55,0.76], [0.27,0.57,0.78],
   [0.28,0.58,0.79], [0.30,0.59,0.80],[0.32,0.61,0.81], [0.34,0.62,0.82], [0.36,0.64,0.84],
   [0.38,0.66,0.85], [0.40,0.67,0.86], [0.43,0.69,0.87],[0.45,0.71,0.89],[0.48,0.72,0.90],
   [0.51,0.74,0.91],[0.54,0.76,0.93],[0.57,0.78,0.94],[0.61,0.80,0.96],[0.64,0.82,0.97]]

In [ ]:
#gurobipy.gurobi.version()

In [ ]:
#diccionario de parámetros

drg={}
for i in range(len(rg)):
    for j in range(len(rg[0])):
        drg[(i+1,j+1)]=rg[i][j]
  
dPo={}
for k in CK:  
    for i in L[k-1]:
       for p in CP:
            dPo[(i,p)]=Po[i-1][p-1]

dH={}
for i in range(len(H)):
    for j in range(len(H[0])):
        dH[(i+1,j+1)]=H[i][j]

damb={}
for k in CK:  
    for i in L[k-1]:
        damb[i]=amb[i-1]
        


dD=dict(zip(CJ,D))
dC=dict(zip(CI,C))


In [ ]:
setSol = {}
sqp=[0.4,0.8]#alpha values
for mm in sqp:
    jjj=mm
    #jjj=0.8
    #variables
    xf={}#people vaccinated by the stationary team in the neighborhoods
    pxf={}
    for k in CK:
        for l in L[k-1]:
            for p in CP:
                for i in CI:
                    for t in CT:
                        xf[l,p,i,t]=m.addVar(vtype=GRB.CONTINUOUS, lb=0 ,name="xf[%s,%s,%s,%s]"%(l,p,i,t))

    qm={}#people vaccinated by the mobile team in the macro zone
    pqm={}
    for k in CK:
        for p in CP:
            for j in CJ:
                qm[k,p,j]=m.addVar(vtype=GRB.CONTINUOUS, lb=0 ,name="qm[%s,%s,%s]"%(k,p,j))

    pxm={}
    xm={}#people vaccinated by the mobile team in neighborhoods
    for k in CK:
        for l in L[k-1]:
            for p in CP:
                for j in CJ:
                    for t in CT:
                        xm[l,p,j,t]=m.addVar(vtype=GRB.CONTINUOUS, lb=0 ,name="xm[%s,%s,%s,%s]"%(l,p,j,t))                             

    pqf={}#people vaccinated by the fixed team in the macrozone
    qf={}
    for k in CK:
        for p in CP:
            for i in CI:
                qf[k,p,i]=m.addVar(vtype=GRB.CONTINUOUS, lb=0 ,name="qf[%s,%s,%s]"%(k,p,i))


    y={}# indicates whether the cell phone has been positioned in one of the following zones
    for k in CK:
        for r in L[k-1]:
            for j in CJ:    
                for t in CT:
                      y[r,j,t]=m.addVar(vtype=GRB.BINARY, lb=0 ,name="y[%s,%s,%s]"%(r,j,t))

    v={}# indicates the population served by the mobile team
    for k in CK:
        for l in L[k-1]:
            for j in CJ:
                for t in CT:
                    for p in CP:
                        v[l,j,t,p]=m.addVar(vtype=GRB.BINARY, lb=0 ,name="v[%s,%s,%s,%s]"%(l,j,t,p))
    sol={}
    for t in CT:
        for p in CP:
            sol[t,p]=m.addVar(vtype=GRB.CONTINUOUS, lb=0 ,name="sol[%s,%s]"%(t,p))

    z={}# number of vaccines allocated to population k
    for k in CK:
        for t in CT:
            z[k,t]=m.addVar(vtype=GRB.CONTINUOUS, lb=0 ,name="z[%s,%s]"%(k,t))    
    #w={}#
    #for j in CJ:
    #   w[j]=m.addVar(vtype=GRB.BINARY, lb=0 ,name="w[%s]"%(j))


    m.update()

    
        
    #4)restricción
    for t in CT:
        print('t: ', str(t), ' of ', str(len(CT)))
        #4)Balance of daily vaccines delivered from the distribution center to the macro-region (communes in our case)
        #m.addConstr(quicksum(z[k,t] for k in CK if (k,t) in z)<= A[t])
        m.addConstr(quicksum(xf[l,p,i,t] for (l,p,i,t1) in 
                         xf if t1==t)
                +quicksum(xm[l,p,j,t] for (l,p,j,t1) in
                          xm if t1==t)<=A[t-1])

    #6)Population of each zone must be served 
    for k in CK:
        for l in L[k-1]:
            for p in CP:
                m.addConstr(quicksum(xf[l,p,i,t] for (l1,p1,i,t) in 
                                     xf if l1==l and p1==p)
                            +quicksum(xm[l,p,j,t] for (l1,p1,j,t) in
                                      xm if l1==l and p1==p)>= 
                                                    dPo[l,p])

    #7)mobile vaccinator capacity
    for t in CT:
        for j in CJ:
            m.addConstr(quicksum(xm[l,p,j,t] for (l,p,j1,t1) in xm if j1==j and t1==t)<=dD[j])

    #8)fixed vaccinator capacity
    for i in CI:
        for t in CT:
            m.addConstr(quicksum(xf[l,p,i,t] for (l,p,i1,t1) in xf if i1==i and t1==t)<=dC[i])    



    #Location restrictions for mobile vaccinators (maximum coverage considering adjacent areas)
    #10)
    for t in CT:
        for k in CK:
            for l in L[k-1]:
                for j in CJ:
                    for p in CP:
                        m.addConstr(quicksum(y[r,j1,t1] for (r,j1,t1) in y if r in N[l] and j1==j and t1==t)>=v[l,j,t,p])                                



    #It is allowed to vaccinate people through a mobile vaccinator as long as it is located in the neighborhoods covered by it.
    #11)restricción
    for k in CK:
        for l in L[k-1]:
            for p in CP:
                for j in CJ:
                    for t in CT:
                        m.addConstr(xm[l,p,j,t] <= v[l,j,t,p]*dPo[l,p])


    #It is decided that it should be installed in at most one neighborhood per day
    #12)restricción
    for j in CJ:
        for t in CT:
            m.addConstr(quicksum(y[r,j,t] for (r,j1,t1) in y if j1==j and t1==t)<=1)

    #the aim is to allow people to be vaccinated in areas adjacent to their neighborhoods.
    for k in CK:
        for l in L[k-1]:
            for p in CP:
                for i in CI:
                    for t in CT:
                        m.addConstr(xf[l,p,i,t] <= dH[i,l]*10000)

    
    
    # sum of the objective function
    for t in CT:
        for p in CP:
            m.addConstr(quicksum(xm[l,p,j,t] for (l,p1,j,t1) in xm if p1==p and t1==t) + 
                        quicksum(xf[s,p,i,t] for (s,p1,i,t1) in xf if p1==p and t1==t)==sol[t,p])

    fo1=quicksum(sol[t,p]*drg[t,p] for (t,p) in sol) #objective function1  
    fo2=quicksum(damb[l]*y[l,j,t]  for (l,j,t) in y) #objective function2

    fn1=(fo1-f1min)/(f1max-f1min)
    fn2=(fo2-f2min)/(f2max-f2min)
    


    m.Params.ObjNumber  = 2
    m.Params.timelimit=3600
    m.Params.MIPGap=0.001
    m.Params.Cuts=3
    m.ModelSense=1 #minimization 
    m.Params.MIPFocus=2
         
    m.setObjectiveN(fn1, 0, weight=jjj, name="f1")
    m.setObjectiveN(fn2, 1, weight=1-jjj, name="f2")
    #m.setObjectiveN(fo3, 2, 1, name="f3")



    print('alpha: ', jjj)


    m.Params.LogFile=str(jjj)+'_ex1.log'
    m.update()

    m.optimize()
    m.write(str(jjj)+"_exp1.sol")
    gv = m.getVars()
    names = m.getAttr('VarName', gv)
    for i in range(m.SolCount):
        m.params.SolutionNumber = i
        xn = m.getAttr('Xn', gv)
        lines = ["{} {}".format(v1, v2) for v1, v2 in zip(names, xn)]
        with open('{}_{}.sol'.format(m.ModelName, i+jjj), 'w') as f:
            f.write("# Solution for model {}\n".format(m.modelName))
            f.write("# Objective value = {}\n".format(m.PoolObjVal))
            f.write("\n".join(lines))

    #m.discardMultiobjEnvs()
    Time_r=m.Runtime


    nSolutions = m.SolCount
    population=sum(sol[t,p].x for (t,p) in sol)

    #m.status = GRB.Status.OPTIMAL    
    #if m.status == GRB.Status.OPTIMAL:    
    if True:
        # Query number of multiple objectives, and number of solutions
        nSolutions  = m.SolCount
        nObjectives = m.NumObj

        for s in range(nSolutions):
         # Set which solution we will query from now on
          m.params.SolutionNumber = s
        # Print objective value of this solution in each objective
        # print('Solution', s, ':', end='')
        for o in range(nObjectives):
        # Set which objective we will query
            m.params.ObjNumber = o
        # Query the o-th objective value
            #print(' ',m.ObjNVal, end='')


        nObjectives = m.NumObj
        #print('Problem has', nObjectives, 'objectives')
        for o in range(nObjectives):
        # Set which objective we will query
            m.params.ObjNumber = o

        # query the full vector of the o-th solution
        #assert m.Status == GRB.Status.OPTIMAL

        # Query number of multiple objectives, and number of solutions
        nSolutions  = m.SolCount
        nObjectives = m.NumObj

    solutions = []


    gaps=[]
    soluciones={}
    sal=[]
    sal = np.zeros((nSolutions, nObjectives))
    for s in range(nSolutions):
    # Set which solution we will query from now on
        m.params.SolutionNumber = s

        # Print objective value of this solution in each objective
        #print('Solution', s, ':', end='')
        for o in range(nObjectives):
            i=o
        # Set which objective we will query
            m.params.ObjNumber = o
        # Query the o-th objective value
            soluciones[(s+1,i+1)]=m.ObjNVal 
            sal[s-1][i-1]=m.ObjNVal

    pas={} # se cálcula la cantidad de personas efectivmente vacunadas tanto en hospitales como ambulancias
    print('Cantidad de personas vacunadas según grupo de riesgo' )
    for k in CK:
        for l in L[k-1]:
            for p in CP:
                pas[l,p]=sum(xf[l,p,i,t].x for (l1,p1,i,t) in xf if l1==l and p1==p)+ sum(xm[l,p,j,t].x for (l1,p1,j,t) in xm if l1==l and p1==p)

    #for p in CP:
     #   for k in CK:
      #      for l in L[k-1]:   
                #print('Barrio:',l, ', Paciente:',p, 'Cantidad:', float(pas[l,p]))

    totalbarrio={}#vaccinated by neighborhood
    totaldembar={}#demand by neighborhood
    nivelser={}#level of service
    totalvac=0 #vaccines delivered to the system 
    for t in CT:
        totalvac=totalvac+ A[t-1]

    totalpob=0
    for k in CK:  
        for i in L[k-1]:
           for p in CP:
            totalpob=totalpob+dPo[(i,p)] #calculate the number of people eligible for vaccination
    print('Vacunas totales :', totalvac )
    print('Población total:', totalpob)
    totalvacunados=0
    totalvacunados=sum(pas[l,p] for (l,p) in pas)#we calculate the number of people actually vaccinated
    print('Personas vacunadas: ', totalvacunados)

    print('Porcentaje de cobertura campaña:', round(100*(totalvac/totalpob),2),'%')
    for k in CK:
        for l in L[k-1]:
            totalbarrio[l]=sum(pas[l,p] for (l1,p) in pas if l1==l)
            totaldembar[l]=sum(dPo[l,p] for (l1,p) in dPo if l1==l)
            print('Porcentaje vacunados por barrio',l,':',round(100*(totalbarrio[l]/totaldembar[l]),2),'%')
            nivelser[l]=round(100*(totalbarrio[l]/totaldembar[l]),2)

    import pandas as pd
    data0 = pd.DataFrame(columns=( 'f1','f2'))

    for i in range(len(sal)):
        for j in range(len(sal[0])):
            data0.loc[i]=[ soluciones[i+1,1],soluciones[i+1,2]] 


    print(data0)
    va={}
    for k in CK:
        for l in L[k-1]:
            for j in CJ:
                for t in CT:
                    for p in CP:
                         va[l,j,t,p]=v[l,j,t,p].x

    xfa={}
    for k in CK:
        for l in L[k-1]:
            for p in CP:
                for i in CI:
                    for t in CT:
                        xfa[l,p,i,t]=xf[l,p,i,t].x
    #calculation of care by hospital
    hos1={}
    hos2={}
    hos3={}
    amb1={}
    amb2={}
    amb3={}
    amb4={}
    amb5={}
    amb6={}
    amb7={}
    amb8={}
    amb9={}
    amb10={}



    for tt in range(1,T+1):
        hos1[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==1)),2)
    #print(hos1)

    for tt in range(1,T+1):
        hos2[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==2)),2)
    #print(hos1)

    for tt in range(1,T+1):
        hos3[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==3)),2)
    #print(hos1)

    for tt in range(1,T+1):
        amb1[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==1)),2)
    #print(amb1)
    for tt in range(1,T+1):
        amb2[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==2)),2)
    #print(amb2)
    for tt in range(1,T+1):
        amb3[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==3)),2)
    #print(amb3)
    for tt in range(1,T+1):
        amb4[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==4)),2)
    #print(amb4)
    for tt in range(1,T+1):
        amb5[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==5)),2)
    #print(amb5)
    for tt in range(1,T+1):
        amb6[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==6)),2)
    #print(amb6)
    for tt in range(1,T+1):
        amb7[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==7)),2)
    #print(amb7)
    for tt in range(1,T+1):
        amb8[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==8)),2)
    #print(amb8)



    import pandas as pd
    data2 = pd.DataFrame(columns=( 'day','h1', 'h2', 'h3','ab1','ab2','ab3','ab4','ab5','ab6','ab7','ab8'))

    for t in CT:
        data2.loc[t]=[t,hos1[t],hos2[t],hos3[t],amb1[t],amb2[t],amb3[t],amb4[t],amb5[t],amb6[t],amb7[t],amb8[t]] 
    print(data2)

    ya={}
    for k in CK:
        for r in L[k-1]:
            for j in CJ:    
                for t in CT:
                    ya[(r,j,t)]=y[r,j,t].x
                    if y[r,j,t].x>0:
                        print(y[r,j,t].x,'dia:',t,'recurso:',j,'barrio:',r)

    for k in CK:
        for l in L[k-1]:
            for j in CJ:
                for t in CT:
                    for p in CP:
                        if v[l,j,t,p].x>0:
                            print(v[l,j,t,p].x, 'el día:',t, 'atiende el barrio',l ,'recurso:',j )
                            
    dia_xa={}
    dia_xb={}
    dia_xc={}
    for tt in range(1,T+1):
        dia_xa[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and p==1)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and p==1))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if p==1)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if p==1)),2)
    print(dia_xa)

    for tt in range(1,T+1):
        dia_xb[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and p==2)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and p==2))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if p==2)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if p==2)),2)
    print(dia_xb)

    for tt in range(1,T+1):
        dia_xc[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and p==3)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and p==3))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if p==3)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if p==3)),2)
    print(dia_xc)


    import pandas as pd
    data = pd.DataFrame(columns=( 'day','A', 'B', 'C'))

    for t in CT:
        data.loc[t]=[t,dia_xa[t],dia_xb[t],dia_xc[t]] 
    print(data)
    dia_q1={}
    dia_q2={}
    dia_q3={}
    dia_q4={}
    for tt in range(1,T+1):
        dia_q1[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<=5)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<=5))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<=5)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<=5)),2)
    print(dia_q1)

    for tt in range(1,T+1):
        dia_q2[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<5 and l<=10)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<5 and l<=10))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<5 and l<=10)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<5 and l<=10)),2)
    print(dia_q2)

    for tt in range(1,T+1):
        dia_q3[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<10 and l<=15)+
                              sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<10 and l<=15))/
                         (sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<10 and l<=15)+
                          sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<10 and l<=15)),2)
    #print(dia_q3)
    for tt in range(1,T+1):
        dia_q4[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<15)+
                              sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<15))/
                         (sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<15)+
                          sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<10)),2)


    import pandas as pd
    data1 = pd.DataFrame(columns=( 'day','k1', 'k2', 'k3','k4'))

    for t in CT:
        data1.loc[t]=[t,dia_q1[t],dia_q2[t],dia_q3[t],dia_q4[t]] 
    print(data1)
    fig = plt.figure()
    x = data['day']
    y = data['A']
    y2 = data['B']
    y3 = data['C']
   
    #error =data['dev std 7']
    #error2 = data['dev std 14']
    #error3 = data['dev std 21']
    pl.plot(x, y, 'k-', label = 'Population group: A',marker="s")
    pl.legend()
    pl.fill_between(x, y-0, alpha = 0.15)
    pl.plot(x, y2, 'k-', label = 'Population group: B', color = 'red',marker="o")
    pl.fill_between(x, y2-0, alpha = 0.15, color = 'red')
    pl.plot(x, y3, 'k-', label = 'Population group: C', color = 'blue',marker="X")
    pl.fill_between(x, y3-0, alpha = 0.15, color='blue')
    
    
    my_xticks = [i for i in range(1,21)] 
    plt.xticks(x, my_xticks)
    plt.ylim(0,100)
    plt.rcParams['figure.figsize'] = [16,7]
    pl.rc('xtick', labelsize=15) 
    pl.rc('ytick', labelsize=15) 
    plt.xlabel('Day ', fontsize=16)
    plt.ylabel('Vaccinated population %', fontsize=16)
    params = {'legend.fontsize': 12,
              'legend.handlelength': 2}
    pl.legend(loc='upper right', bbox_to_anchor=(0.2, 1.00), shadow=True, ncol=1)
    plt.rcParams.update(params)
    pl.show()


    #plt.figure(figsize=(14,7))
    fig = plt.figure()
    x = data['day']
    y = data['A']
    y2 = data['B']
    y3 = data['C']
    
    #error =data['dev std 7']
    #error2 = data['dev std 14']
    #error3 = data['dev std 21']
    pl.plot(x, y, 'k-', label = 'Population group: A',marker="s")
    pl.legend()
    pl.fill_between(x, y-0, alpha = 0.15)
    pl.plot(x, y2, 'k-', label = 'Population group: B', color = 'red',marker="o")
    pl.fill_between(x, y2-0, alpha = 0.15, color = 'red')
    pl.plot(x, y3, 'k-', label = 'Population group: C', color = 'blue',marker="X")
    pl.fill_between(x, y3-0, alpha = 0.15, color='blue')
   
    
    my_xticks = [i for i in range(1,21)] 
    plt.xticks(x, my_xticks)
    plt.ylim(0,100)
    plt.rcParams['figure.figsize'] = [16,7]
    pl.rc('xtick', labelsize=15) 
    pl.rc('ytick', labelsize=15) 
    plt.xlabel('Day ', fontsize=16)
    plt.ylabel('Vaccinated population %', fontsize=16)
    params = {'legend.fontsize': 12,
              'legend.handlelength': 2}
    pl.legend(loc='upper right', bbox_to_anchor=(0.2, 1.00), shadow=True, ncol=1)
    plt.rcParams.update(params)
    pl.show()


    fig = plt.figure()
    x = data1['day']
    y = data1['k1']
    y2 = data1['k2']
    y3 = data1['k3']
    y4=data1['k4']



    pl.plot(x, y, 'k-', label = 'Macrozone 1',marker="s")
    pl.legend()
    pl.fill_between(x, y-0, alpha = 0.15)
    pl.plot(x, y2, 'k-', label = 'Macrozone 2', color = 'red',marker="o")
    pl.fill_between(x, y2-0, alpha = 0.15, color = 'red')
    pl.plot(x, y3, 'k-', label = 'Macrozone 3', color = 'blue',marker="X")
    pl.fill_between(x, y3-0, alpha = 0.15, color='blue')
    pl.plot(x, y4, 'k-', label = 'Macrozone 4', color = 'green',marker="P")
    pl.fill_between(x, y4-0, alpha = 0.15, color='green')
    
    my_xticks = [i for i in range(1,21)] 
    plt.xticks(x, my_xticks)
    plt.ylim(0,100)
    plt.rcParams['figure.figsize'] = [16,7]
    pl.rc('xtick', labelsize=10) 
    pl.rc('ytick', labelsize=10) 
    plt.xlabel('Day ', fontsize=16)
    plt.ylabel('Vaccinated population %', fontsize=16)
    params = {'legend.fontsize': 12,
              'legend.handlelength': 2}
    pl.legend(loc='upper right', bbox_to_anchor=(0.2, 1.00), shadow=True, ncol=1)
    plt.rcParams.update(params)
    pl.show()
    
    
    
        #calculation of care by hospital
    hos1={}
    hos2={}
    hos3={}

    for tt in range(1,T+1):
        hos1[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==1)),0)
    print(hos1)

    for tt in range(1,T+1):
        hos2[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==2)),0)
    print(hos1)

    for tt in range(1,T+1):
        hos3[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==3)),0)
    print(hos1)



    import pandas as pd
    data2 = pd.DataFrame(columns=( 'day','h1', 'h2', 'h3'))

    for t in CT:
        data2.loc[t]=[t,hos1[t],hos2[t],hos3[t]] 
    print(data2)
    from openpyxl import Workbook

    #workbook_name="Hoja1.xlsx"
    #wb = load_workbook(workbook_name)
    wb = Workbook()
    ws1 = wb.create_sheet("F.O's")
    ws2 = wb.create_sheet("Plan_vacunación")
    ws3 = wb.create_sheet("Plan_móvil")
    ws4 = wb.create_sheet("Plan_fijo")
    ws5 = wb.create_sheet("Tiempo_Gap")
    ws6 = wb.create_sheet("Plan_movil_detalle")
    
    #saving the value of the f.o's in excel

    for i in range(len(sal)):
        for j in range(len(sal[0])):
                ws1.cell(row = 1, column = 1, value = "f1")
                ws1.cell(row = i+2, column = 1, value = soluciones[i+1,1])
                ws1.cell(row = 1, column = 2, value = "f2")
                ws1.cell(row = i+2, column = 2, value = soluciones[i+1,2])
    #vaccination plan
    for t in CT:
        ws2.cell(row = 1, column = 1, value = "día")
        ws2.cell(row = t+1, column = 1, value = t)
        ws2.cell(row = 1, column = 2, value = "CF1")
        ws2.cell(row = t+1, column = 2, value = hos1[t])
        ws2.cell(row = 1, column = 3, value = "CF2")
        ws2.cell(row = t+1, column = 3, value = hos2[t])
        ws2.cell(row = 1, column = 4, value = "CF3")
        ws2.cell(row = t+1, column = 4, value = hos3[t])
        ws2.cell(row = 1, column = 5, value = "CM1")
        ws2.cell(row = t+1, column = 5, value = amb1[t])             
        ws2.cell(row = 1, column = 6, value = "CM2") 
        ws2.cell(row = t+1, column = 6, value = amb2[t]) 
        ws2.cell(row = 1, column = 7, value = "CM3")
        ws2.cell(row = t+1, column = 7, value = amb3[t]) 
        ws2.cell(row = 1, column = 8, value = "CM4")             
        ws2.cell(row = t+1, column = 8, value = amb4[t] )    
        ws2.cell(row = 1, column = 9, value = "CM5") 
        ws2.cell(row = t+1, column = 9, value = amb5[t])
        ws2.cell(row = 1, column = 10, value = "CM6")
        ws2.cell(row = t+1, column = 10, value = amb6[t])
        ws2.cell(row = 1, column = 11, value = "CM7")
        ws2.cell(row = t+1, column = 11, value = amb7[t])
        ws2.cell(row = 1, column = 12, value = "CM8")
        ws2.cell(row = t+1, column = 12, value = amb8[t])
    
    #mobile vaccination plan
    cont=1
    for k in CK:
        for r in L[k-1]:
            for j in CJ:    
                for t in CT:
                    if ya[r,j,t]>0:
                        cont=cont+1   
                        ws3.cell(row = 1, column = 1, value = "Día")
                        ws3.cell(row = 1, column = 2, value = "Centro Móvil")
                        ws3.cell(row = 1, column = 3, value = "Se instala en") 
                        ws3.cell(row = cont+1, column = 1, value = t)
                        ws3.cell(row = cont+1, column = 2, value = j)
                        ws3.cell(row = cont+1, column = 3, value = r)
    cont=1
    for k in CK:
        for l in L[k-1]:
            for j in CJ:
                for t in CT:
                    for p in CP:
                        if va[l,j,t,p]>0:
                            cont=cont+1

                            ws3.cell(row = 1, column = 4, value = "Día")
                            ws3.cell(row = 1, column = 5, value = "Centro Móvil")
                            ws3.cell(row = 1, column = 6, value = "Barrio")
                            ws3.cell(row = 1, column = 7, value = "Grupo")

                            ws3.cell(row = cont+1, column = 4, value = t)
                            ws3.cell(row = cont+1, column = 5, value = j)
                            ws3.cell(row = cont+1, column = 6, value = l)
                            ws3.cell(row = cont+1, column = 7, value = p)
                            
    cont=1
    for k in CK:
        for l in L[k-1]:
            for p in CP:
                for i in CI:
                    for t in CT:
                        if xfa[l,p,i,t]>0:
                            cont=cont+1
                            ws4.cell(row = 1, column = 1, value = "Día")
                            ws4.cell(row = 1, column = 2, value = "CF")
                            ws4.cell(row = 1, column = 3, value = "Barrio")
                            ws4.cell(row = 1, column = 4, value = "Grupo")
                            ws4.cell(row = 1, column = 5, value = "Cantidad")

                            ws4.cell(row = cont+1, column = 1, value = t)
                            ws4.cell(row = cont+1, column = 2, value = i)
                            ws4.cell(row = cont+1, column = 3, value = l)
                            ws4.cell(row = cont+1, column = 4, value = p)
                            ws4.cell(row = cont+1, column = 5, value = xfa[l,p,i,t])
    
    ws5.cell(row = 2, column = 2, value = "Tiempo")
    ws5.cell(row = 3, column = 2, value = Time_r)
    cont=1
    for k in CK:
        for l in L[k-1]:
            for p in CP:
                for j in CJ:
                    for t in CT:
                        if float(xm[l,p,j,t].x) >0:
                            cont=cont+1
                        
                            ws6.cell(row = 1, column = 1, value = "Día")
                            ws6.cell(row = 1, column = 2, value = "Movil")
                            ws6.cell(row = 1, column = 3, value = "Barrio")
                            ws6.cell(row = 1, column = 4, value = "Grupo")
                            ws6.cell(row = 1, column = 5, value = "Cantidad")


                            ws6.cell(row = cont+1, column = 1, value = t)
                            ws6.cell(row = cont+1, column = 2, value = j)
                            ws6.cell(row = cont+1, column = 3, value = l)
                            ws6.cell(row = cont+1, column = 4, value = p)
                            ws6.cell(row = cont+1, column = 5, value = xm[l,p,j,t].x)
                        
    
    
    # Save the file
    namexl="compacto"+str(jjj)+".xlsx"
    wb.save(namexl)




    
    

### 

In [ ]:
pas={} # se cálcula la cantidad de personas efectivmente vacunadas tanto en hospitales como ambulancias
print('Cantidad de personas vacunadas según grupo de riesgo' )
for k in CK:
    for l in L[k-1]:
        for p in CP:
            pas[l,p]=sum(xf[l,p,i,t].x for (l1,p1,i,t) in xf if l1==l and p1==p)+ sum(xm[l,p,j,t].x for (l1,p1,j,t) in xm if l1==l and p1==p)

#for p in CP:
 #   for k in CK:
  #      for l in L[k-1]:   
            #print('Barrio:',l, ', Paciente:',p, 'Cantidad:', float(pas[l,p]))

totalbarrio={}#vacunados por barrios
totaldembar={}#demanda por barrios
nivelser={}#nivel de servicio
totalvac=0 #vacunas entregadas al sistema 
for t in CT:
    totalvac=totalvac+ A[t-1]

totalpob=0
for k in CK:  
    for i in L[k-1]:
       for p in CP:
        totalpob=totalpob+dPo[(i,p)] #cálculamos la cantidad de personas dísponibles a vacunar
print('Vacunas totales :', totalvac )
print('Población total:', totalpob)
totalvacunados=0
totalvacunados=sum(pas[l,p] for (l,p) in pas)#calculamos las personas realmente vacunadas
print('Personas vacunadas: ', totalvacunados)

print('Porcentaje de cobertura campaña:', round(100*(totalvac/totalpob),2),'%')
for k in CK:
    for l in L[k-1]:
        totalbarrio[l]=sum(pas[l,p] for (l1,p) in pas if l1==l)
        totaldembar[l]=sum(dPo[l,p] for (l1,p) in dPo if l1==l)
        print('Porcentaje vacunados por barrio',l,':',round(100*(totalbarrio[l]/totaldembar[l]),2),'%')
        nivelser[l]=round(100*(totalbarrio[l]/totaldembar[l]),2)

import pandas as pd
data0 = pd.DataFrame(columns=( 'f1','f2'))

for i in range(len(sal)):
    for j in range(len(sal[0])):
        data0.loc[i]=[ soluciones[i+1,1],soluciones[i+1,2]] 


print(data0)

In [ ]:
va={}
for k in CK:
    for l in L[k-1]:
        for j in CJ:
            for t in CT:
                for p in CP:
                     va[l,j,t,p]=v[l,j,t,p].x

xfa={}
for k in CK:
    for l in L[k-1]:
        for p in CP:
            for i in CI:
                for t in CT:
                    xfa[l,p,i,t]=xf[l,p,i,t].x

In [ ]:
#calculo de atención por hospitales
hos1={}
hos2={}
hos3={}
amb1={}
amb2={}
amb3={}
amb4={}
amb5={}
amb6={}
amb7={}
amb8={}
amb9={}
amb10={}



for tt in range(1,T+1):
    hos1[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==1)),2)
#print(hos1)

for tt in range(1,T+1):
    hos2[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==2)),2)
#print(hos1)

for tt in range(1,T+1):
    hos3[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==3)),2)
#print(hos1)

for tt in range(1,T+1):
    amb1[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==1)),2)
#print(amb1)
for tt in range(1,T+1):
    amb2[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==2)),2)
#print(amb2)
for tt in range(1,T+1):
    amb3[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==3)),2)
#print(amb3)
for tt in range(1,T+1):
    amb4[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==4)),2)
#print(amb4)
for tt in range(1,T+1):
    amb5[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==5)),2)
#print(amb5)
for tt in range(1,T+1):
    amb6[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==6)),2)
#print(amb6)
for tt in range(1,T+1):
    amb7[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==7)),2)
#print(amb7)
for tt in range(1,T+1):
    amb8[tt]=round((sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t==tt and j==8)),2)
#print(amb8)



import pandas as pd
data2 = pd.DataFrame(columns=( 'day','h1', 'h2', 'h3','ab1','ab2','ab3','ab4','ab5','ab6','ab7','ab8'))

for t in CT:
    data2.loc[t]=[t,hos1[t],hos2[t],hos3[t],amb1[t],amb2[t],amb3[t],amb4[t],amb5[t],amb6[t],amb7[t],amb8[t]] 
print(data2)

ya={}
for k in CK:
    for r in L[k-1]:
        for j in CJ:    
            for t in CT:
                ya[(r,j,t)]=y[r,j,t].x
                if y[r,j,t].x>0:
                    print(y[r,j,t].x,'dia:',t,'recurso:',j,'barrio:',r)
        
for k in CK:
    for l in L[k-1]:
        for j in CJ:
            for t in CT:
                for p in CP:
                    if v[l,j,t,p].x>0:
                        print(v[l,j,t,p].x, 'el día:',t, 'atiende el barrio',l ,'recurso:',j )

In [ ]:

dia_xa={}
dia_xb={}
dia_xc={}
for tt in range(1,T+1):
    dia_xa[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and p==1)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and p==1))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if p==1)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if p==1)),2)
print(dia_xa)

for tt in range(1,T+1):
    dia_xb[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and p==2)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and p==2))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if p==2)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if p==2)),2)
print(dia_xb)

for tt in range(1,T+1):
    dia_xc[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and p==3)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and p==3))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if p==3)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if p==3)),2)
print(dia_xc)


import pandas as pd
data = pd.DataFrame(columns=( 'day','A', 'B', 'C'))

for t in CT:
    data.loc[t]=[t,dia_xa[t],dia_xb[t],dia_xc[t]] 
print(data)

In [ ]:
dia_q1={}
dia_q2={}
dia_q3={}
for tt in range(1,T+1):
    dia_q1[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<=5)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<=5))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<=5)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<=5)),2)
print(dia_q1)

for tt in range(1,T+1):
    dia_q2[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<5 and l<=10)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<5 and l<=10))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<5 and l<=10)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<5 and l<=10)),2)
print(dia_q2)

for tt in range(1,T+1):
    dia_q3[tt]=round(100*(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t<=tt and l<10)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if t<=tt and l<10))/(sum(xf[l,p,i,t].x for (l,p,i,t) in xf if l<10)+sum(xm[l,p,j,t].x for (l,p,j,t) in xm if l<10)),2)
#print(dia_q3)


import pandas as pd
data1 = pd.DataFrame(columns=( 'day','k1', 'k2', 'k3'))

for t in CT:
    data1.loc[t]=[t,dia_q1[t],dia_q2[t],dia_q3[t]] 
print(data1)

In [ ]:
fig = plt.figure()
x = data['day']
y = data['A']
y2 = data['B']
y3 = data['C']
#error =data['dev std 7']
#error2 = data['dev std 14']
#error3 = data['dev std 21']
pl.plot(x, y, 'k-', label = 'Population group: A',marker="s")
pl.legend()
pl.fill_between(x, y-0, alpha = 0.15)
pl.plot(x, y2, 'k-', label = 'Population group: B', color = 'red',marker="o")
pl.fill_between(x, y2-0, alpha = 0.15, color = 'red')
pl.plot(x, y3, 'k-', label = 'Population group: C', color = 'blue',marker="X")
pl.fill_between(x, y3-0, alpha = 0.15, color='blue')
my_xticks = [i for i in range(1,21)] 
plt.xticks(x, my_xticks)
plt.ylim(0,100)
plt.rcParams['figure.figsize'] = [16,7]
pl.rc('xtick', labelsize=15) 
pl.rc('ytick', labelsize=15) 
plt.xlabel('Day ', fontsize=16)
plt.ylabel('Vaccinated population %', fontsize=16)
params = {'legend.fontsize': 12,
          'legend.handlelength': 2}
pl.legend(loc='upper right', bbox_to_anchor=(0.2, 1.00), shadow=True, ncol=1)
plt.rcParams.update(params)
pl.show()


#plt.figure(figsize=(14,7)) 

In [ ]:
fig = plt.figure()
x = data['day']
y = data['A']
y2 = data['B']
y3 = data['C']
#error =data['dev std 7']
#error2 = data['dev std 14']
#error3 = data['dev std 21']
pl.plot(x, y, 'k-', label = 'Population group: A',marker="s")
pl.legend()
pl.fill_between(x, y-0, alpha = 0.15)
pl.plot(x, y2, 'k-', label = 'Population group: B', color = 'red',marker="o")
pl.fill_between(x, y2-0, alpha = 0.15, color = 'red')
pl.plot(x, y3, 'k-', label = 'Population group: C', color = 'blue',marker="X")
pl.fill_between(x, y3-0, alpha = 0.15, color='blue')
my_xticks = [i for i in range(1,21)] 
plt.xticks(x, my_xticks)
plt.ylim(0,100)
plt.rcParams['figure.figsize'] = [16,7]
pl.rc('xtick', labelsize=15) 
pl.rc('ytick', labelsize=15) 
plt.xlabel('Day ', fontsize=16)
plt.ylabel('Vaccinated population %', fontsize=16)
params = {'legend.fontsize': 12,
          'legend.handlelength': 2}
pl.legend(loc='upper right', bbox_to_anchor=(0.2, 1.00), shadow=True, ncol=1)
plt.rcParams.update(params)
pl.show()

In [ ]:
fig = plt.figure()
x = data1['day']
y = data1['k1']
y2 = data1['k2']
y3 = data1['k3']
y4=data1['k4']



pl.plot(x, y, 'k-', label = 'Macrozone 1',marker="s")
pl.legend()
pl.fill_between(x, y-0, alpha = 0.15)
pl.plot(x, y2, 'k-', label = 'Macrozone 2', color = 'red',marker="o")
pl.fill_between(x, y2-0, alpha = 0.15, color = 'red')
pl.plot(x, y3, 'k-', label = 'Macrozone 3', color = 'blue',marker="X")
pl.fill_between(x, y3-0, alpha = 0.15, color='blue')
my_xticks = [i for i in range(1,21)] 
plt.xticks(x, my_xticks)
plt.ylim(0,100)
plt.rcParams['figure.figsize'] = [16,7]
pl.rc('xtick', labelsize=10) 
pl.rc('ytick', labelsize=10) 
plt.xlabel('Day ', fontsize=16)
plt.ylabel('Vaccinated population %', fontsize=16)
params = {'legend.fontsize': 12,
          'legend.handlelength': 2}
pl.legend(loc='upper right', bbox_to_anchor=(0.2, 1.00), shadow=True, ncol=1)
plt.rcParams.update(params)
pl.show()

In [ ]:
#calculo de atención por hospitales
hos1={}
hos2={}
hos3={}

for tt in range(1,T+1):
    hos1[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==1)),0)
print(hos1)

for tt in range(1,T+1):
    hos2[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==2)),0)
print(hos1)

for tt in range(1,T+1):
    hos3[tt]=round((sum(xf[l,p,i,t].x for (l,p,i,t) in xf if t==tt and i==3)),0)
print(hos1)



import pandas as pd
data2 = pd.DataFrame(columns=( 'day','h1', 'h2', 'h3'))

for t in CT:
    data2.loc[t]=[t,hos1[t],hos2[t],hos3[t]] 
print(data2)



In [ ]:
from openpyxl import Workbook

#workbook_name="Hoja1.xlsx"
#wb = load_workbook(workbook_name)
wb = Workbook()
ws1 = wb.create_sheet("F.O's")
ws2 = wb.create_sheet("Plan_vacunación")
ws3 = wb.create_sheet("Plan_móvil")
ws4 = wb.create_sheet("Plan_fijo")
ws5 = wb.create_sheet("Tiempo_Gap")



In [ ]:
#guardando el valor de las f.o's en excel

for i in range(len(sal)):
    for j in range(len(sal[0])):
            ws1.cell(row = 1, column = 1, value = "f1")
            ws1.cell(row = i+2, column = 1, value = soluciones[i+1,1])
            ws1.cell(row = 1, column = 2, value = "f2")
            ws1.cell(row = i+2, column = 2, value = soluciones[i+1,2])

In [ ]:
soluciones

In [ ]:
#plan de vacunación
for t in CT:
    ws2.cell(row = 1, column = 1, value = "día")
    ws2.cell(row = t+1, column = 1, value = t)
    ws2.cell(row = 1, column = 2, value = "CF1")
    ws2.cell(row = t+1, column = 2, value = hos1[t])
    ws2.cell(row = 1, column = 3, value = "CF2")
    ws2.cell(row = t+1, column = 3, value = hos2[t])
    ws2.cell(row = 1, column = 4, value = "CF3")
    ws2.cell(row = t+1, column = 4, value = hos3[t])
    ws2.cell(row = 1, column = 5, value = "CM1")
    ws2.cell(row = t+1, column = 5, value = amb1[t])             
    ws2.cell(row = 1, column = 6, value = "CM2") 
    ws2.cell(row = t+1, column = 6, value = amb2[t]) 
    ws2.cell(row = 1, column = 7, value = "CM3")
    ws2.cell(row = t+1, column = 7, value = amb3[t]) 
    ws2.cell(row = 1, column = 8, value = "CM4")             
    ws2.cell(row = t+1, column = 8, value = amb4[t] )    
    ws2.cell(row = 1, column = 9, value = "CM5") 
    ws2.cell(row = t+1, column = 9, value = amb5[t])
    ws2.cell(row = 1, column = 10, value = "CM6")
    ws2.cell(row = t+1, column = 10, value = amb6[t])
    ws2.cell(row = 1, column = 11, value = "CM7")
    ws2.cell(row = t+1, column = 11, value = amb7[t])
    ws2.cell(row = 1, column = 12, value = "CM8")
    ws2.cell(row = t+1, column = 12, value = amb8[t])


In [ ]:
#plan de vacunación móvil
cont=1
for k in CK:
    for r in L[k-1]:
        for j in CJ:    
            for t in CT:
                if ya[r,j,t]>0:
                    cont=cont+1   
                    ws3.cell(row = 1, column = 1, value = "Día")
                    ws3.cell(row = 1, column = 2, value = "Centro Móvil")
                    ws3.cell(row = 1, column = 3, value = "Se instala en") 
                    ws3.cell(row = cont+1, column = 1, value = t)
                    ws3.cell(row = cont+1, column = 2, value = j)
                    ws3.cell(row = cont+1, column = 3, value = r)
cont=1
for k in CK:
    for l in L[k-1]:
        for j in CJ:
            for t in CT:
                for p in CP:
                    if va[l,j,t,p]>0:
                        cont=cont+1
                        
                        ws3.cell(row = 1, column = 4, value = "Día")
                        ws3.cell(row = 1, column = 5, value = "Centro Móvil")
                        ws3.cell(row = 1, column = 6, value = "Barrio")
                        ws3.cell(row = 1, column = 7, value = "Grupo")
                        
                        ws3.cell(row = cont+1, column = 4, value = t)
                        ws3.cell(row = cont+1, column = 5, value = j)
                        ws3.cell(row = cont+1, column = 6, value = l)
                        ws3.cell(row = cont+1, column = 7, value = p)
                        
                        




In [ ]:
cont=1
for k in CK:
    for l in L[k-1]:
        for p in CP:
            for i in CI:
                for t in CT:
                    if xfa[l,p,i,t]>0:
                        cont=cont+1
                        ws4.cell(row = 1, column = 1, value = "Día")
                        ws4.cell(row = 1, column = 2, value = "CF")
                        ws4.cell(row = 1, column = 3, value = "Barrio")
                        ws4.cell(row = 1, column = 4, value = "Grupo")
                        ws4.cell(row = 1, column = 5, value = "Cantidad")
                        
                        ws4.cell(row = cont+1, column = 1, value = t)
                        ws4.cell(row = cont+1, column = 2, value = i)
                        ws4.cell(row = cont+1, column = 3, value = l)
                        ws4.cell(row = cont+1, column = 4, value = p)
                        ws4.cell(row = cont+1, column = 5, value = xfa[l,p,i,t])

In [ ]:
ws5.cell(row = 2, column = 2, value = "Tiempo")
ws5.cell(row = 3, column = 2, value = Time_r)
# Save the file
namexl="output"+str(jjj)+".xlsx"
wb.save(namexl)